In [ ]:
exist = !which ffmpeg
if not exist:
  !curl https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz -o ffmpeg.tar.xz \
     && tar -xf ffmpeg.tar.xz && rm ffmpeg.tar.xz
  ffmdir = !find . -iname ffmpeg-*-static
  path = %env PATH
  path = path + ':' + ffmdir[0]
  %env PATH $path
!which ffmpeg
!pip install -r requirements.txt
print("설치가 완료되었습니다!")

In [ ]:
from playlist_generator import playlist

# 아래의 큰 따옴표 안에 들어있는 문자를 파일 이름으로 바꾸고 프로그램을 실행하세요.
excel_file_name = "지우고 수정해야 될 부분"
playlist.generate(excel_file_name)